In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pandas as pd

# SparkContext是Spark的入口，相当于应用程序的main函数，http://codingdict.com/article/8882
sc = SparkContext()   
sql_sc = SQLContext(sc)

pd_df_ratings = pd.read_csv('ratings_small.csv')
pyspark_df_ratings = sql_sc.createDataFrame(pd_df_ratings)
pyspark_df_ratings = pyspark_df_ratings.drop('Timestamp')

# 创建ALS模型
# rank隐分类 3，迭代次数10，正则项0。1
als = ALS(rank=3,maxIter=10,regParam=0.1,userCol='userId',itemCol='movieId',ratingCol='rating')
model = als.fit(pyspark_df_ratings)

# 对userId=100进行Top-N推荐
recommendations = model.recommendForAllUsers(5)
print(recommendations.where(recommendations.userId==100).collect())

[Row(userId=100, recommendations=[Row(movieId=67504, rating=5.237862586975098), Row(movieId=83318, rating=5.237862586975098), Row(movieId=83411, rating=5.237862586975098), Row(movieId=83359, rating=5.237862586975098), Row(movieId=4731, rating=5.148364543914795)])]
